
<a id='numerical-linear-algebra'></a>

# Direct Numerical Methods for Linear Algebra

## Contents

- [Direct Numerical Methods for Linear Algebra](#Direct-Numerical-Methods-for-Linear-Algebra)  
  - [Overview](#Overview)  
  - [Simple Examples](#Simple-Examples)  
  - [Factorizations](#Factorizations)  
  - [Calculating the QR Decomposition](#Calculating-the-QR-Decomposition)  
  - [Banded Matrices](#Banded-Matrices)  

> You cannot learn too much linear algebra. – Benedict Gross

## Overview

In particular, we will examine the structure of matrices and linear operators (e.g., dense, sparse, symmetric, tridiagonal, banded) and
discuss how it can be exploited to radically increase the performance of solving the problems.

We build on [linear algebra](linear_algebra.html) and [orthogonal projections](orth_proj.html).

The methods in this section are called direct methods, and they are qualitatively similar to performing Gaussian elimination to factor matrices and solve systems.  In [iterative methods and sparsity](iterative_methods_sparsity.html) we examine a different approach with iterative algorithms, and generalized the matrices as linear operators.

The list of specialized packages for these tasks is enormous and growing, but some of the important organizations to
look at are [JuliaMatrices](https://github.com/JuliaMatrices) , [JuliaSparse](https://github.com/JuliaSparse), and [JuliaMath](https://github.com/JuliaMath)

**NOTE** This lecture explores techniques linear algebra, with an emphasis on large systems.  You may wish to review multiple-dispatch and generic programming in  introduction to types, and consider further study on [generic programming](more_julia/generic_programming.html).

### Setup

In [1]:
using LinearAlgebra, Statistics, BenchmarkTools, SparseArrays, Random
Random.seed!(42);  # seed random numbers for reproducibility

### Applications

Some key questions to motivate the lecture.  Is the following a computationally expensive operation as the size of the matrix increases?

- Multiplying two matrices?  It depends.  Multiplying 2 diagonals is trivial.  
- Solving a linear system of equations?  It depends.  If the matrix is the identity, the solution is the vector itself.  
- Finding the eigenvalues of a matrix?  It depends.  The eigenvalues of a triangular matrix are the diagonal.  


With that in mind, in this section lecture, we consider variations on three classic problems.

First is the solution to

$$
A x = b
$$

for a square $ A $ where we will maintain throughout there is a unique solution.

On paper, since the [Invertible Matrix Theorem](https://en.wikipedia.org/wiki/Invertible_matrix#The_invertible_matrix_theorem) tells us a unique solution is
equivalent to $ A $ being invertible, we often write the solution as

$$
x = A^{-1} b
$$

Second, in the case of a rectangular matrix, $ A $ we consider the [linear least-squares](https://en.wikipedia.org/wiki/Linear_least_squares) solution
to

$$
\min_x ||Ax -b||^2
$$

From theory, we know that $ A $ has linearly independent columns that the solution is the [normal equations](https://en.wikipedia.org/wiki/Linear_least_squares#Derivation_of_the_normal_equations)

$$
x = (A'A)^{-1}A'b
$$

And finally, consider the eigenvalue problem of finding $ x $ and $ \lambda $ such that

$$
A x = \lambda x
$$

For the eigenvalue problems.  Keep in mind that that you do not always require all of the $ \lambda $, and sometimes the largest (or smallest) would be enough.  For example, calculating the spectral radius only requires the maximum eigenvalue in absolute value.

The theme of this lecture, and numerical linear algebra in general, comes down to three principles:

1. **identify structure** (e.g. [symmetric, sparse, diagonal,etc.](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/index.html#Special-matrices-1)) of $ A $ in order to use **specialized algorithms**  
1. **do not lose structure** by applying the wrong linear algebra operations at the wrong times (e.g. sparse matrix becoming dense)  
1. understand the **computational complexity** of each algorithm  

### Computational Complexity

As the goal of this section is to move towards numerical linear algebra of large systems, we need to understand how well algorithms scale with size.  This notion is called [computational complexity](https://en.wikipedia.org/wiki/Computational_complexity).

While this notion of complexity can work at various levels such as the number of [significant digits](https://en.wikipedia.org/wiki/Computational_complexity_of_mathematical_operations#Arithmetic_functions) for basic mathematical operations, the amount of memory and storage required, or the amount of time) - but we will typically focus on the time-complexity.

For time-complexity, the `N` is usually the dimensionality of the problem, although occasionally the key will be the number of non-zeros in the matrix or width of bands.  For our applications, time-complexity is best thought of as the number of floating point operations (e.g. add, multiply, etc.) required.

Complexity of algorithms is typically written in [Big O](https://en.wikipedia.org/wiki/Big_O_notation) notation which provides bounds on the scaling.

Formally, we can write this as $ f(N) = O(g(N)) \text{ as} N \to \infty $ wher the interpretation is that there exists some constants $ M, N_0 $ such that

$$
f(N) \leq M g(N), \text{ for } N > N_0
$$

For example, the complexity of finding an LU Decomposition of a dense matrix is $ O(N^3) $ which should be read as there being a constant where
eventually the number of floating point operations required decompose a matrix of size $ N\times N $ grows cubically.

Keep in mind that these are asymptotic results intended to understanding the scaling of the problem, and the constant can matter for a given
fixed size.

For example, the number of operations required for an [LU decomposition](https://en.wikipedia.org/wiki/LU_decomposition#Algorithms) of a dense $ N \times N $ matrix $ 2/3 N^3 $, ignoring the $ N^2 $ and lower terms.  However, sparse matrix algorithms instead scale with the number of non-zeros in the matrix instead.

### Rules of Computational Complexity

When combining algorithms, you will sometimes need to think through how [combining algorithms](https://en.wikipedia.org/wiki/Big_O_notation#Properties) changes complexity.  For example, if you do,

1. an $ O(N^3) $ operation $ P $ times, then it simply changes the constant and remains $ O(N^3) $  
1. one $ O(N^3) $ operation and another $ O(N^2) $ one, then you take the max, which does not change the scale $ O(N^3) $  
1. a repetition of a $ O(N) $ operation that itself uses an $ O(N) $ one, you take the product, and the complexity becomes $ O(N^2) $  


With this, there is a word of caution: dense matrix-multiplication is an [expensive operation](https://en.wikipedia.org/wiki/Computational_complexity_of_mathematical_operations#Matrix_algebra) for unstructured matrices, and the basic version is $ O(N^3) $.

Of course, modern libraries use highly turned and [careful algorithms](https://en.wikipedia.org/wiki/Matrix_multiplication_algorithm) to multiply matrices and exploit the computer architecture, memory cache, etc., but this simply lowers the constant of proportionality and they remain $ O(N^3) $.

A consequence is that, since many algorithms require matrix-matrix multiplication, it means that it is usually not possible to go below that order without further matrix structure.

That is, changing the constant of proportionality for a given size can help, but in order to achieve higher scaling you need to identify matrix structure and ensure your operations do not lose it.

### Losing Structure

As a first example of a structured matrix, consider a [sparse arrays](https://docs.julialang.org/en/v1/stdlib/SparseArrays/index.html)

In [2]:
A = sprand(10, 10, 0.45)  # random 10x10, 45% filled with non-zeros

@show nnz(A)  # counts the number of non-zeros
invA = sparse(inv(Array(A)))  # Julia won't even invert sparse directly
@show nnz(invA);

nnz(A) = 47
nnz(invA) = 100


This increase from 45 to 100 percent dense demonstrates that significant sparsity can be lost when calculating an inverse.

The results can be even more extreme.  Consider a tridiagonal matrix of size $ N \times N $
that might come out of a Markov Chain or discretization  of a diffusion process,

In [3]:
N = 5
A = Tridiagonal([fill(0.1, N-2); 0.2], fill(0.8, N), [0.2; fill(0.1, N-2);])

5×5 Tridiagonal{Float64,Array{Float64,1}}:
 0.8  0.2   ⋅    ⋅    ⋅ 
 0.1  0.8  0.1   ⋅    ⋅ 
  ⋅   0.1  0.8  0.1   ⋅ 
  ⋅    ⋅   0.1  0.8  0.1
  ⋅    ⋅    ⋅   0.2  0.8

The number of non-zeros here is approximately $ 3 N $, linear, which scales well for huge matrices into the millions or billions

But consider the inverse

In [4]:
inv(A)

5×5 Array{Float64,2}:
  1.29099      -0.327957     0.0416667  -0.00537634   0.000672043
 -0.163978      1.31183     -0.166667    0.0215054   -0.00268817 
  0.0208333    -0.166667     1.29167    -0.166667     0.0208333  
 -0.00268817    0.0215054   -0.166667    1.31183     -0.163978   
  0.000672043  -0.00537634   0.0416667  -0.327957     1.29099    

Now, the matrix is fully dense and scales $ N^2 $

This also applies to the $ A' A $ operation in the normal equations of LLS.

In [5]:
A = sprand(20, 21, 0.3)
@show nnz(A)/20^2
@show nnz(A'*A)/21^2;

nnz(A) / 20 ^ 2 = 0.2825
nnz(A' * A) / 21 ^ 2 = 0.800453514739229


While there is some variation based on the randoms chosen, we see that a 30 percent dense matrix becomes almost full dense
after the product is taken.

**Sparsity/Structure is not just for storage**:  While we have been emphasizing counting the non-zeros as a heuristic, the primary reason to maintain structure
and sparsity is not for using less memory to store the matrices.

Size can sometimes become important (e.g. a 1 million by 1 million tridiagonal matrix needs to store 3 million numbers (i.e, about 6MB of memory), where a dense one requires 1 trillion (i.e., about 1TB of memory).

But, as we will see, the main purpose of considering sparsity and matrix structure is that it enables specialized algorithms which typically
have a lower-computational order than unstructured dense, or even an unstructured sparse operations.

First, create convenient functions for benchmarking which displays the type

In [6]:
using BenchmarkTools
function benchmark_solve(A, b)
    println("A\\b for typeof(A) = $(string(typeof(A)))")
    @btime $A \ $b
end

benchmark_solve (generic function with 1 method)

Then, take away structure to see the impact on performance,

In [7]:
N = 1000
b = rand(N)
A = Tridiagonal([fill(0.1, N-2); 0.2], fill(0.8, N), [0.2; fill(0.1, N-2);])
A_sparse = sparse(A)
A_dense = Array(A)

# benchmark solution to system A x = b and A * B
benchmark_solve(A, b)
benchmark_solve(A_sparse, b)
benchmark_solve(A_dense, b);

A\b for typeof(A) = Tridiagonal{Float64,Array{Float64,1}}
  19.700 μs (9 allocations: 47.75 KiB)
A\b for typeof(A) = SparseMatrixCSC{Float64,Int64}
  532.799 μs (69 allocations: 1.06 MiB)
A\b for typeof(A) = Array{Float64,2}
  26.550 ms (5 allocations: 7.65 MiB)


This example shows what is at stake:  using a structured tridiagonal may be 10-20x faster than using a sparse matrix which is 100x faster then
using a dense matrix.

In fact, the difference becomes more extreme as the matrices grow.  Solving a tridiagonal system is an $ O(N) $ while that of a dense matrix without any structure is $ O(N^3) $.  The complexity of a sparse solution is more complicated, but roughly scales by $ O(nnz(N)) $, i.e. the number of nonzeros.

## Simple Examples

### Inverting Matrices

To begin, consider a simple linear system of a dense matrix

In [8]:
N = 4
A = rand(N,N)
b = rand(N)

4-element Array{Float64,1}:
 0.5125312396664188
 0.6828679275339775
 0.3454193757566306
 0.8706954827549864

On paper, we to solve for $ A x = b $ by inverting the matrix,

In [9]:
x = inv(A) * b

4-element Array{Float64,1}:
 -0.8230616861035331
  1.1336956414831805
 -1.642325023331038 
  2.2104913647288558

As we will see, inverting matrices should be used for theory, not for code.  The classic advice that you should [never invert a matrix](https://www.johndcook.com/blog/2010/01/19/dont-invert-that-matrix) may be [slightly exaggerated](https://arxiv.org/abs/1201.6035), but is generally good advice.  In fact, the methods used by libraries to invert matrices typically calculate the same factorizations used for computing a system of equations.

To summarize the wisdom: solving a system by inverting a matrix is always a little slower, potentially less accurate, and will often lose crucial sparsity.

### Triangular Matrices and Back/Forward Substitution

To begin, consider solving a system with an `UpperTriangular` matrix,

In [10]:
b = [1.0, 2.0, 3.0]
U = UpperTriangular([1.0 2.0 3.0; 0.0 5.0 6.0; 0.0 0.0 9.0])

3×3 UpperTriangular{Float64,Array{Float64,2}}:
 1.0  2.0  3.0
  ⋅   5.0  6.0
  ⋅    ⋅   9.0

This system is especially easy to solve using [back-substitution](https://en.wikipedia.org/wiki/Triangular_matrix#Forward_and_back_substitution).  In particular, $ x_3 = b_3 / U_{33}, x_2 = (b_2 - x_3 U_{23})/U_{22} $, etc.

In [11]:
U \ b

3-element Array{Float64,1}:
 0.0               
 0.0               
 0.3333333333333333

A `LowerTriangular` has similar properties and can be solved with forward-substitution.  For these matrices, no further matrix factorization is needed.

### A Warning on Matrix Multiplication

Why we write matrix multiplications in our algebra with abandon, in practice the operation scales very poorly without any matrix structure.

Matrix multiplication is so important to modern computers that the constant of scaling in front of the scaling has been radically reduced
when using a proper package, but the order is still $ O(N^3) $ in practice.

Sparse matrix multiplication, on the other hand, is $ O(N M_A M_B) $ where $ M_A $ are the number of nonzeros per row of $ A $ and $ B $ are the number of non-zeros per column of $ B $.

By the rules of computational order, that means any algorithm this means that any algorithm requiring a matrix multiplication of dense matrices requires at least $ O(N^3) $ operation.

The other important question is what is the structure of the resulting matrix. As always, we want to avoid losing

For example, multiplying an upper triangular by a lower triangular

In [12]:
N = 5
U = UpperTriangular(rand(N,N))

5×5 UpperTriangular{Float64,Array{Float64,2}}:
 0.409653  0.0195282  0.87974     0.552949  0.252486
  ⋅        0.436677   0.662624    0.736553  0.224344
  ⋅         ⋅         0.00653798  0.649019  0.134132
  ⋅         ⋅          ⋅          0.168964  0.834989
  ⋅         ⋅          ⋅           ⋅        0.708384

In [13]:
L = U'

5×5 Adjoint{Float64,UpperTriangular{Float64,Array{Float64,2}}}:
 0.409653   0.0       0.0         0.0       0.0     
 0.0195282  0.436677  0.0         0.0       0.0     
 0.87974    0.662624  0.00653798  0.0       0.0     
 0.552949   0.736553  0.649019    0.168964  0.0     
 0.252486   0.224344  0.134132    0.834989  0.708384

But the multiplication is fully dense (e.g. think of a cholesky multiplied by itself to produce a covariance matrix)

In [14]:
L * U

5×5 Array{Float64,2}:
 0.167816    0.00799978  0.360388  0.226517  0.103432
 0.00799978  0.191068    0.306533  0.332434  0.102897
 0.360388    0.306533    1.21306   0.978752  0.371655
 0.226517    0.332434    0.978752  1.29804   0.532991
 0.103432    0.102897    0.371655  0.532991  1.33109 

On the other hand, a tridiagonal times a diagonal is still a tridiagonal

In [15]:
A = Tridiagonal([fill(0.1, N-2); 0.2], fill(0.8, N), [0.2; fill(0.1, N-2);])
D = Diagonal(rand(N))
D * A

5×5 Tridiagonal{Float64,Array{Float64,1}}:
 0.561922   0.14048     ⋅          ⋅          ⋅      
 0.0568224  0.454579   0.0568224   ⋅          ⋅      
  ⋅         0.0402454  0.321963   0.0402454   ⋅      
  ⋅          ⋅         0.01826    0.14608    0.01826 
  ⋅          ⋅          ⋅         0.0123203  0.049281

## Factorizations

When you tell a numerical analyst you are solving a linear system directly, their first question is “which factorization?”


<a id='jl-decomposition'></a>

### LU Decomposition

For a general dense matrix without any other structure (i.e. not known to be symmetric, tridiagonal, etc.) the standard approach is to
factor the matrix iin order to exploit the speed of backward and forward substitution to complete the solution.

The $ LU $ decompositions finds a lower triangular $ L $ and upper triangular $ U $ such that $ L U = A $.

We can see which algorithm Julia will use for the `\` operator by looking at the `factorize` function for a given
matrix.

In [16]:
N = 4
A = rand(N,N)
b = rand(N)

Af = factorize(A)  # chooses the right factorization, LU here

LU{Float64,Array{Float64,2}}
L factor:
4×4 Array{Float64,2}:
 1.0       0.0       0.0       0.0
 0.933578  1.0       0.0       0.0
 0.613678  0.551771  1.0       0.0
 0.686831  0.729438  0.932112  1.0
U factor:
4×4 Array{Float64,2}:
 0.916223  0.0915758  0.221304   0.0908984
 0.0       0.712447   0.083115   0.681078 
 0.0       0.0        0.806706   0.239325 
 0.0       0.0        0.0       -0.452328 

In this case, it provides an $ L $ and $ U $ factorization (with [pivoting](https://en.wikipedia.org/wiki/LU_decomposition#LU_factorization_with_full_pivoting) ).

With the factorization complete, we can solve different `b` right hand sides

In [17]:
b2 = rand(N)
Af \ b2

4-element Array{Float64,1}:
  0.06909902818646778
 -0.453743557563131  
 -0.11556849117372223
  1.4306380720047653 

This decomposition also includes a $ P $ is a [permutation matrix](https://en.wikipedia.org/wiki/Permutation_matrix) such
that $ P A = L U $.

In [18]:
Af.P * A ≈ Af.L * Af.U

true

We can also directly calculate an `lu` decomposition without the pivoting,

In [19]:
L, U = lu(A, Val(false))  # the Val(false) provides solution without permutation matrices

LU{Float64,Array{Float64,2}}
L factor:
4×4 Array{Float64,2}:
 1.0         0.0      0.0     0.0
 0.735697    1.0      0.0     0.0
 0.657339   16.8663   1.0     0.0
 1.07115   171.134   10.8358  1.0
U factor:
4×4 Array{Float64,2}:
 0.855366   0.79794       0.28972    0.765939
 0.0       -0.00445928    0.751419  -0.233514
 0.0        0.0         -11.8757     4.10594 
 0.0        0.0           0.0       -5.25829 

And we can verify the decomposition

In [20]:
A ≈ L * U

true

To see roughly how the solver works, note that we can write the problem $ A x = b $ as $ L U x = b $.  Let $ U x = y $, which breaks the
problem into two sub-problems.

$$
L y = b
U x = y
$$

To demonstrate this, we can solve it by first using

In [21]:
y = L \ b

4-element Array{Float64,1}:
  0.49627820582486404
  0.2520008871120075 
 -4.428091208429694  
  5.119647290772903  

In [22]:
x = U \ y
x ≈ A \ b  # Check identical

true

The LU decomposition also has specialized algorithms for structured matrices, such as a Tridiagonal

In [23]:
N = 1000
b = rand(N)
A = Tridiagonal([fill(0.1, N-2); 0.2], fill(0.8, N), [0.2; fill(0.1, N-2);])
factorize(A) |> typeof

LU{Float64,Tridiagonal{Float64,Array{Float64,1}}}

This factorization is the key to the performance of the `A \ b`

Finally, just as a dense matrix without any structure will tend to use a LU decomposition to solve systems,
so will a sparse matrix

In [24]:
A_sparse = sparse(A)
factorize(A_sparse) |> typeof  # dropping the tridiagonal structure to just become sparse

SuiteSparse.UMFPACK.UmfpackLU{Float64,Int64}

In [25]:
benchmark_solve(A, b)
benchmark_solve(A_sparse, b);

A\b for typeof(A) = Tridiagonal{Float64,Array{Float64,1}}
  26.200 μs (9 allocations: 47.75 KiB)
A\b for typeof(A) = SparseMatrixCSC{Float64,Int64}
  575.199 μs (69 allocations: 1.06 MiB)


### Cholesky Decomposition

For real, symmetric, positive definitive matrices, a Cholesky decomposition is a specialized version of the LU decomposition where $ L = U' $.

The Cholesky is directly useful on its own (e.g. [Classical Control with Linear Algebra](time_series_models/classical_filtering.html)) but it is also an efficient factorization to solve symmetric positive definite system.

As always, symmetry allows specialized algorithms.

In [26]:
N = 500
B = rand(N,N)
A_dense = B' * B  # an easy way to generate a symmetric positive definite matrix
A = Symmetric(A_dense)

factorize(A) |> typeof

BunchKaufman{Float64,Array{Float64,2}}

Here, the $ A $ decomposition is [Bunch-Kaufman](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/index.html#LinearAlgebra.bunchkaufman) rather than a
Cholesky, because Julia doesn’t know the matrix is positive definite.  We can manually factorize with a cholesky,

In [27]:
cholesky(A) |> typeof

Cholesky{Float64,Array{Float64,2}}

Benchmarking,

In [28]:
b = rand(N)
benchmark_solve(A, b)
benchmark_solve(A_dense, b)
@btime cholesky($A, check=false) \ $b;

A\b for typeof(A) = Symmetric{Float64,Array{Float64,2}}
  55.539 ms (8 allocations: 2.16 MiB)
A\b for typeof(A) = Array{Float64,2}
  42.937 ms (5 allocations: 1.92 MiB)
  8.863 ms (7 allocations: 1.91 MiB)


## Calculating the QR Decomposition

[Previously](orth_proj.html#qr-decomposition) , we learned about applications of the QR application to solving the linear least squares.


<dl style='margin: 20px 0;'>
<dt>While in principle, the solution to least-squares is $ x = (A'A)^{-1}A'b $, in practice note that $ A'A $ becomes very dense and inverse are</dt>
<dd>
rarely a good idea.

</dd>

</dl>

The QR decomposition is a decomposition $ A = Q R $ where $ Q $ is an orthogonal matrix (i.e. $ Q'Q = Q Q' = I $) and $ R $ is
a upper triangular matrix.

Given the  [previous derivation](orth_proj.html#qr-decomposition) we showed that the, given the decomposition, we can write the least squares problem as
the solution to

$$
R x = Q' b
$$

Where, as discussed above, the upper-triangular structure of $ R $ can be solved easily with back substitution.

For Julia, the `\` operator will solve this problem whenever the given `A` is rectangular

In [34]:
N = 10
M = 3
x_true = rand(3)

A = rand(N,M) .+ randn(N)
b = rand(N)
x = A \ b

3-element Array{Float64,1}:
  0.210701802805989   
  0.023316560374275572
 -0.061369824702479765

To manually use the QR decomposition: **Note** the real code would be more subtle

In [35]:
Af = qr(A)
Q = Af.Q
R = [Af.R; zeros(N - M, M)] # Stack with zeros
@show Q * R ≈ A
x = R \ Q'*b  # the QR way

Q * R ≈ A = true


3-element Array{Float64,1}:
  0.21070180280598896
  0.02331656037427554
 -0.0613698247024797 

This stacks the `R` with zeros to multiple, but the more specialized algorithm would not multiply directly
in that way.

In some cases, if an LU is not available for a particular matrix structure, the QR factorization
can also be used to solve systems of equations (i.e. not just LLS).  This tends to be about 2x slower than the LU,
but is of the same computational order.

To see this,

In [31]:
N = 5
A = rand(N,N)
b = rand(N)
@show A \ b
@show qr(A) \ b;

A \ b = [-2.5121616885875326, -2.2885498462395253, 4.900473250988739, 4.108181179194703, -3.334611050458787]
qr(A) \ b = [-2.512161688587532, -2.288549846239526, 4.900473250988739, 4.108181179194702, -3.3346110504587867]


## Banded Matrices